# Data Preprocessing and Cleaning

Data cleaning is an essential step in the process of preparing data for further processing and serving into a large language model. Without data cleaning, the language model would be limited in its ability to interpret and process the data accurately. Data cleaning helps to ensure that the data is in a consistent format, free of any inconsistencies or errors that could lead to incorrect results. It also helps to reduce the amount of time needed to process the data, as it eliminates the need to manually check for any inconsistencies or errors. Additionally, data cleaning can help to improve the accuracy of the language model by removing any irrelevant or incorrect data. By taking the time to properly clean the data, the language model can more accurately interpret and process the data, leading to better results.

> 📍 Fill out the missing pieces in the source source to get everything working (indicated by `#FIXME`).

## Count tokens

> **Important note**
>
> *You do not need to manually tokenize strings before feeding texts into the model. This will be done automatically once you put instructions into the `prompt` parameter. However, you can use the `tiktoken` library to check how a string is tokenized and count the numbers of tokens to calculate the cost of an API call. Learn more [here](https://platform.openai.com/docs/introduction/tokens).*

Tokenizing text strings is an important step in natural language processing (NLP) as it helps models like GPT-3 understand the structure of a text string. Tokenizing breaks a text string into smaller pieces called tokens, which can then be analyzed and used by the model. By understanding the structure of a text string, models can better understand the meaning of the text. Additionally, tokenizing helps to determine the cost of an Azure OpenAI Service API call, as usage is priced by token. Furthermore, different models use different encodings, so it is important to tokenize text strings in the appropriate format.

`tiktoken` supports three encodings used by Azure OpenAI Service models:

| Encoding name | Azure OpenAI Service models |
| ------------- | -------------- |
| gpt2 (or r50k_base) | Most GPT-3 models |
| p50k_base | Code models, text-davinci-002, text-davinci-003 |
| cl100k_base | text-embedding-ada-002 |

Tokens in English typically range from one character to one word (e.g. "t" or "great"), though some languages may have tokens that are shorter or longer than one character or word. Spaces are usually placed at the beginning of words (e.g. " is" instead of "is " or "+"is"). You can use the Tokenizer to quickly check how a string is tokenized.

To show it briefly, we will use `tiktoken` to tokenize a text string and see how the output looks like.

In [0]:
%pip install tiktoken

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import tiktoken

encoding = tiktoken.get_encoding("p50k_base")
encoding.encode("Hello world, this is fun!")

Out[1]: [15496, 995, 11, 428, 318, 1257, 0]

Write a script that shows the string tokens from an input phrase.

In [0]:
[encoding.decode_single_token_bytes(token) for token in encoding.encode("Hello world, this is fun!")]

Out[2]: [b'Hello', b' world', b',', b' this', b' is', b' fun', b'!']

Let's write a function to count the number of tokens in a text string.

In [0]:
def get_num_tokens_from_string(string: str, encoding_name: str='p50k_base') -> int:
    """Returns the number of tokens in a text by a given encoding."""
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(string))

get_num_tokens_from_string("Hello World, this is fun!")

Out[3]: 7

## Clean data

Next we'll perform some light data cleaning by removing redundant whitespace and cleaning up the punctuation to prepare the data for tokenization. Again, tokenization is not required for the model to work, but it is a good practice to do so to ensure that the data is in a consistent format and that the model is able to process the data correctly. Also, it makes sure that the request is not too long for the model as the maximum number of tokens for davinci is 2048, e.g., equivalent to around 2-3 pages of text.

> **Best Practices**
> - **Replace newlines with a single space**: Unless you're embedding code, we suggest replacing newlines (\n) in your input with a single space, as we have observed inferior results when newlines are present.

In [0]:
import re

def normalize_text(string, sep_token = " \n "):
    """Normalize text by removing unnecessary characters and altering the format of words."""
    # make text lowercase
    string = re.sub(r'\s+',  ' ', string).strip()
    string = re.sub(r". ,","",string)
    # remove all instances of multiple spaces
    string = string.replace("..",".")
    string = string.replace(". .",".")
    string = string.replace("\n", "")
    string = string.strip()
    return string

Generate some data to test the cleaning function.

In [0]:
normalize_text("Hola que tal") #FIXME

Out[5]: 'Hola que tal'

## Ejercicio 1: Validar longitud de texto antes de enviar a OpenAI

Escribe una función llamada validate_text_length que valide si un texto supera un límite de tokens especificado. Si el texto supera el límite, la función debe dividirlo en segmentos de tamaño adecuado para enviarlos al modelo de OpenAI. Usa la codificación cl100k_base.



In [0]:
import tiktoken

def validate_text_length(text, max_tokens=2048, encoding_name='cl100k_base'):
    # Cargar el codificador especificado (cl100k_base por defecto)
    encoding = tiktoken.get_encoding(encoding_name)
    
    # Tokenizar el texto
    tokens = encoding.encode(text)
    
    # Verificar si la longitud del texto (en tokens) excede el límite
    if len(tokens) > max_tokens:
        print(f"El texto excede el límite de {max_tokens} tokens. Se dividirá en segmentos.")
        
        # Dividir el texto en segmentos de tamaño adecuado
        segments = []
        for i in range(0, len(tokens), max_tokens):
            segment = tokens[i:i + max_tokens]
            segment_text = encoding.decode(segment)
            segments.append(segment_text)
        
        return segments
    else:
        print(f"El texto no excede el límite de {max_tokens} tokens.")
        return [text]

# Prueba la función
text = "Lorem ipsum " * 500  # Texto largo
segments = validate_text_length(text)  # Aquí se llama la función

# Verifica el número de segmentos y su longitud
print(len(segments))  # Resultado esperado: Más de 1 segmento

# Verifica el número de tokens en el primer segmento
print(len(tiktoken.get_encoding('cl100k_base').encode(segments[0])))  # <= 2048 tokens


El texto no excede el límite de 2048 tokens.
1
1001


## Ejercicio 2: Filtrar palabras prohibidas en un texto

Escribe una función llamada filter_prohibited_words que remueva palabras prohibidas de un texto antes de enviarlo a OpenAI. 

Como ejemplo queremos Eliminar palabras como: ['password', 'confidential', 'secret']. 



In [0]:
import re

def filter_prohibited_words(text, prohibited_words):
    # Crear una expresión regular para encontrar las palabras prohibidas
    prohibited_pattern = r'\b(' + '|'.join(map(re.escape, prohibited_words)) + r')\b'
    
    # Reemplazar las palabras prohibidas por [REDACTED]
    cleaned_text = re.sub(prohibited_pattern, '[REDACTED]', text, flags=re.IGNORECASE)
    
    return cleaned_text

# Prueba la función
text = "This document contains password and secret information."
prohibited_words = ['password', 'confidential', 'secret']

cleaned_text = filter_prohibited_words(text, prohibited_words)  # Llamada a la función
print(cleaned_text)  # Resultado esperado: "This document contains [REDACTED] and [REDACTED] information."


This document contains [REDACTED] and [REDACTED] information.


## Ejercicio 3: Identificar idioma del texto

Crea una función detect_language que use langdetect para identificar el idioma de un texto y devuelva el código del idioma (e.g., en, es). Verifica el idioma antes de enviarlo al modelo OpenAI. Debes instalar la libreria langdetect


In [0]:
%pip install langdetect

Python interpreter will be restarted.
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=7b876fc25e3f2ae25a19f4695ece84e3eed69758981e012f446257f937aa22a0
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect
Python interpreter will be restarted.


In [0]:
from langdetect import detect

def detect_language(text):
    """Detecta el idioma de un texto."""
    # Usamos langdetect para detectar el idioma del texto
    return detect(text)

# Prueba la función
textEN = "This is futbol"
textEs= "Este texto es en español"
language = detect_language(text)  # Llamada a la función
print(language)  # Resultado esperado: 'es' (para español)


en


## Ejercicio 5: Generar prompts efectivos

Escribe una función generate_effective_prompt que genere un prompt optimizado para OpenAI, dado un tema y un objetivo. Usa plantillas como:
“Explain [tema] in simple terms for a [audiencia].”



In [0]:
def generate_effective_prompt(topic, audience="general audience"):
    """Genera un prompt optimizado para OpenAI."""
    # Generar el prompt usando la plantilla indicada
    prompt = f"Explain {topic} in simple terms for a {audience}."
    return prompt

# Prueba la función
topic = "machine learning"
prompt = generate_effective_prompt(topic)  # Llamada a la función
print(prompt)  # Resultado esperado: "Explain machine learning in simple terms for a general audience."





## Ejercicio 6: Medir costo estimado del uso de OpenAI

Crea una función estimate_cost que calcule el costo estimado de una solicitud a OpenAI basado en el número de tokens. Supón un costo de $0.02 por 1000 tokens.



In [0]:
import tiktoken

def estimate_cost(text, encoding_name='cl100k_base', cost_per_1k_tokens=0.02):
    """Calcula el costo estimado de un texto basado en el número de tokens."""
    # Cargar el codificador especificado
    encoding = tiktoken.get_encoding(encoding_name)
    
    # Tokenizar el texto
    tokens = encoding.encode(text)
    
    # Calcular el número de tokens
    num_tokens = len(tokens)
    
    # Calcular el costo estimado
    cost = (num_tokens / 1000) * cost_per_1k_tokens
    return cost
text = "Cuantos sera el coste de esto?" * 100  # Texto de prueba
cost = estimate_cost(text)  # Llamada a la función
print(f"El costo estimado es: ${cost:.4f}")  # Resultado esperado: Costo aproximado basado en la cantidad de tokens




## Ejercicio 7: Limpiar texto JSON para OpenAI

Escribe una función clean_json_text que tome un JSON como entrada y limpie todas las claves y valores de caracteres no ASCII.


In [0]:
import json
import re

def clean_json_text(json_obj):
    """Limpia texto JSON eliminando caracteres no ASCII."""
    
    # Función auxiliar para limpiar los valores
    def clean_value(value):
        # Si el valor es un string, eliminamos los caracteres no ASCII
        if isinstance(value, str):
            return re.sub(r'[^\x00-\x7F]', '', value)
        # Si el valor es un diccionario, limpiamos recursivamente
        elif isinstance(value, dict):
            return clean_json_text(value)
        # Si el valor es una lista, limpiamos cada elemento
        elif isinstance(value, list):
            return [clean_value(v) for v in value]
        # Si es otro tipo de dato, lo devolvemos tal cual
        return value

    # Limpiar todas las claves y valores del JSON
    return {k: clean_value(v) for k, v in json_obj.items()}

# Prueba la función
data = {"key1": "Hello 😊", "key2": "Café and thé", "key3": {"nestedKey": "Texto con símbolo ⚡"}}
cleaned_data = clean_json_text(data)
print(cleaned_data)  
# Resultado esperado: {"key1": "Hello ", "key2": "Caf and th", "key3": {"nestedKey": "Texto con símbolo "}}
